In [1]:
import pickle
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image as keras_image
import gradio as gr
import os
import openai
import cv2

## 測試gpt api

In [2]:
with open('./data/texes.txt', 'r', encoding='utf-8') as fh:
    tmp = fh.read()
    itemlist = tmp.split(',')

itemlist = str(itemlist)

In [3]:
keyfile = open("key.txt", "r")
key = keyfile.readline()

openai.api_key = key

In [2]:
# start_idx = 0
# result = ''
# while start_idx < len(itemlist):
#     end_idx = min(start_idx + 1600, len(itemlist))
#     sub_list = itemlist[start_idx:end_idx]
#     response = openai.ChatCompletion.create(
#         model="ada",
#         messages=[
#             {"role": "system", "content": "You are a chatbot"},
#             {"role": "user", "content": f"提供以上文字之繁體中文摘要與英文翻譯：{sub_list}"}
#         ]
#     )  
#     for choice in response.choices:
#         result += choice.message.content
#     print(start_idx)
#     start_idx = end_idx

# with open('output.txt', 'w', encoding='utf-8') as output_file:
#     output_file.write(result)

## 處理已訓練模組

In [2]:
loaded_model = load_model("model_epoch_06.h5", compile=False)
model_config = loaded_model.get_config()
#loaded_model.summary()

2023-11-02 13:45:41.738417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def pred(img):

    # Resize the image to match the input shape of your model
    img = img.resize((256, 256))  # Adjust the size as needed
    # Convert the image to a NumPy array
    img_np = np.array(img)
    
    # Preprocess the image to match the model's input requirements
    img_np = keras_image.img_to_array(img_np)
    
    img_for_plot = img_np / 255.0  # Normalize the image if necessary
    
    img_np = np.expand_dims(img_np, axis=0)  # Add a batch dimension
    
    # Make predictions
    predictions = loaded_model.predict(img_np)
    
    return img_for_plot, predictions

def image_mod(img):
    
    image, predictions = pred(img)

    print(predictions)
    
    result = ""
    
    if predictions[0][0] < 0.5:

        result = "fake image"
    else :
        
        result = "true image"

    return result

def chatbtn(content):
    
    print(content)
    
    return content

def display_image_from_video(video):

    capture_image = cv.VideoCapture(video) 
    # 獲取視頻中的一幀，變數分別為是否成功讀取了一幀以及捕獲的圖像幀
    ret, frame = capture_image.read()

    img = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    img = cv.resize(img, (256, 256))  # 調整大小
    img_for_plot = img / 255.0  # 正規化圖像
    
    img_np = np.expand_dims(img, axis=0)  # Add a batch dimension
    
#     # Make predictions
#     predictions = loaded_model.predict(img_np)
    
#     result = ""
    
#     if predictions[0][0] < 0.5:

#         result = "fake image"
#     else :
        
#         result = "true image"

#     return result
    return img_for_plot

In [18]:
cam_port = 0
cam = cv2.VideoCapture(cam_port) 
# reading the input using the camera 
result, image = cam.read() 
# If image will detected without any error,  
# show result 
if result: 
    # showing result, it take frame name and image  
    # output 
    cv2.imshow("images/GeeksForGeeks", image) 
    # saving image in local storage 
    cv2.imwrite("images/GeeksForGeeks.png", image) 
    # If keyboard interrupt occurs, destroy image  
    # window 
    # cv2.waitKey(0) 
    # cv2.destroyWindow("GeeksForGeeks") 
else: 
    print("No image detected. Please! try again") 

In [19]:
title = """<h1 align="center">AI臉部辨識🚀</h1>"""
                
with gr.Blocks(css = """.gradio-container {background-color: #3f7791}""") as demo:
    gr.HTML(title)
    gr.HTML('''<center><a href="https://github.com/kennywang112?tab=repositories" alt="GitHub Repo"></a></center>''')
    with gr.Row():
        
        with gr.Column(elem_id = "col_container"):
            
            chatbot = gr.Chatbot(elem_id='chatbot')
            inputs = gr.Textbox(placeholder= "Hi there!", label= "Type an input and press Enter")
            state = gr.State([])
            btn = gr.Button('ask') 
            
        btn.click(chatbtn, [inputs], [])

        with gr.Column(elem_id = "col_container"):
            
            gr.Interface(
                image_mod,
                gr.Image(type="pil"),
                gr.components.Text(),
                examples=[
                    os.path.join("images/real_face0.jpeg"),
                    os.path.join("images/real_face1.jpeg"),
                    os.path.join("images/fake_face1.jpeg"),
                    os.path.join("images/GeeksForGeeks.png"),
                ],
            )
                
        with gr.Column(elem_id = "col_container"):
            
            inputs = gr.components.Video()
            outputs = gr.components.Text()
            # outputs = gr.components.Image()
            update = gr.Button('判斷影片圖像')

            update.click(display_image_from_video, inputs = [inputs], outputs = [outputs])
    
demo.queue().launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 29ms/step
[[1. 0.]]
Keyboard interruption in main thread... closing server.
